In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from sklearn.decomposition import PCA
import sklearn as skl

# For loading huggingface stuff
from sentence_transformers import SentenceTransformer
from extractors.chartok import CharacterTok

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Feature Extraction

In [7]:
# Possible values: 
# labse - makes use of LaBSE for multilingual sentence embeddings
# char - makes use of a character-level tokenization scheme.
FEATURE_EXTRACTOR = "char" 

In [8]:
# Load the dataset from the corresponding csv

DATASET_PATHS = ["data/training", "data/validation", "data/test"]


In [9]:
# Load the feature extractor specified in embedding generator
feature_extractor = None 

if FEATURE_EXTRACTOR == "labse":
    # LaBSE found here: https://huggingface.co/sentence-transformers/LaBSE
    feature_extractor = SentenceTransformer('sentence-transformers/LaBSE')
elif FEATURE_EXTRACTOR == "char":
    feature_extractor = CharacterTok()
else: 
    pass 

In [10]:
for db_path in DATASET_PATHS:
    loaded_df = pd.read_csv(db_path + ".csv")

    # Prepare sentence list
    sentences = loaded_df["text"]

    # Apply the feature extractor to a target dataset

    encodings = []

    if FEATURE_EXTRACTOR == "labse":
        feature_extractor : SentenceTransformer = feature_extractor
        encodings = feature_extractor.encode(sentences)
    elif FEATURE_EXTRACTOR == "char":
        feature_extractor : CharacterTok = feature_extractor 
        encodings = feature_extractor.encode(sentences)
    else: 
        pass

    # Store individual floats in the vector in their own columns.

    transposed = encodings.T
    new_df = loaded_df.copy(deep=True)

    for i in range(0, len(encodings[0])):
        new_df['_e' + str(i)] = transposed[i]

    # Save the feature list to a new dataset
    NEW_FILE_PATH = db_path + "_" + FEATURE_EXTRACTOR
    new_df.to_csv(NEW_FILE_PATH + ".csv", index=False)

ValueError: Length of values (23) does not match length of index (16000)